In [46]:
import pandas as pd

# Creating the dataframe
data = {
    'height': [86, 102, 104, 99, 97, 97, 109, 104, 99, 91, 97, 99, 107, 104, 97, 97, 102, 99, 97, 104],
    'stem_biomass': [351.214, 327.133, 263.747, 436.844, 358.629, 400.098, 421.424, 655.545, 367.028, 424.114, 295.702, 558.76, 480.61, 459.416, 291.962, 391.503, 461.778, 644.39, 488.287, 425.033],
    'agw': [30.693, 49.118, 25.473, 29.237, 26.486, 28.547, 24.763, 23.882, 29.215, 28.785, 28.779, 26.71, 23.864, 25.387, 29.463, 28.609, 29.259, 20.889, 27.726, 27.658],
    'gn': [14488, 11556, 10389, 23492, 22889, 20322, 16820, 42889, 17564, 20569, 15636, 32545, 23712, 30183, 15980, 20807, 20719, 34173, 23543, 28911],
    'grain_yield': [2844.291, 2710.761, 2604.396, 3757.808, 2393.898, 3373.228, 4163.714, 6199.606, 2480.774, 3433.071, 2776.903, 5549.672, 5440.748, 4420.801, 2982.743, 3825.262, 4487.73, 6445.997, 4661.089, 4329.462]
}

df= pd.DataFrame(data)
df.head(5)

,height,stem_biomass,agw,gn,grain_yield
0,86,351.214,30.693,14488,2844.291
1,102,327.133,49.118,11556,2710.761
2,104,263.747,25.473,10389,2604.396
3,99,436.844,29.237,23492,3757.808
4,97,358.629,26.486,22889,2393.898


In [47]:
# Splitting the dataset into training and testing sets
from sklearn.model_selection import train_test_split, cross_val_score

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

In [48]:
train_df

,height,stem_biomass,agw,gn,grain_yield
11,99,558.760,26.710,32545,5549.672
3,99,436.844,29.237,23492,3757.808
18,97,488.287,27.726,23543,4661.089
16,102,461.778,29.259,20719,4487.730
13,104,459.416,25.387,30183,4420.801
2,104,263.747,25.473,10389,2604.396
9,91,424.114,28.785,20569,3433.071
19,104,425.033,27.658,28911,4329.462
4,97,358.629,26.486,22889,2393.898
12,107,480.610,23.864,23712,5440.748


In [49]:
test_df

,height,stem_biomass,agw,gn,grain_yield
0,86,351.214,30.693,14488,2844.291
17,99,644.390,20.889,34173,6445.997
15,97,391.503,28.609,20807,3825.262
1,102,327.133,49.118,11556,2710.761
8,99,367.028,29.215,17564,2480.774
5,97,400.098,28.547,20322,3373.228


In [50]:
# RandomForest Model

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [51]:
# Training the model with cross-validation
scores = cross_val_score(rf, train_df.drop(columns='grain_yield'), train_df['grain_yield'], cv=4, n_jobs=-1, verbose=1)
print("Cross-validation scores:", scores)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Cross-validation scores: [-0.6494757   0.42576691  0.44707789  0.33874885]


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.5s finished


In [52]:
# Fitting the model
rf.fit(train_df.drop(columns='grain_yield'), train_df['grain_yield'])

RandomForestRegressor()

In [53]:
# Summarizing the model
importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': train_df.drop(columns='grain_yield').columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)
print(feature_importance_df)

        feature  importance
1  stem_biomass    0.668120
3            gn    0.187989
2           agw    0.079414
0        height    0.064477


In [54]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Predicting and calculating RMSE on training set
train_predictions = rf.predict(train_df.drop(columns='grain_yield'))
train_rmse = np.sqrt(mean_squared_error(train_df['grain_yield'], train_predictions))
print("Train RMSE:", train_rmse)

# Predicting and calculating RMSE on testing set
test_predictions = rf.predict(test_df.drop(columns='grain_yield'))
test_rmse = np.sqrt(mean_squared_error(test_df['grain_yield'], test_predictions))
print("Test RMSE:", test_rmse)

Train RMSE: 241.5627309704843
Test RMSE: 486.02897379552


In [55]:
# Permutation importance

from sklearn.inspection import permutation_importance

result = permutation_importance(rf, test_df.drop(columns='grain_yield'), test_df['grain_yield'], n_repeats=10, random_state=42, n_jobs=-1)
sorted_idx = result.importances_mean.argsort()
importance_df = pd.DataFrame({
    'feature': test_df.drop(columns='grain_yield').columns[sorted_idx],
    'importance': result.importances_mean[sorted_idx]
}).sort_values(by='importance', ascending=False)

print(importance_df)

        feature  importance
3  stem_biomass    0.923494
2            gn    0.105701
1           agw    0.062976
0        height    0.003199


In [56]:
# Importing Libraries:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

# Creating the dataframe
data = {
    'height': [86, 102, 104, 99, 97, 97, 109, 104, 99, 91, 97, 99, 107, 104, 97, 97, 102, 99, 97, 104],
    'stem_biomass': [351.214, 327.133, 263.747, 436.844, 358.629, 400.098, 421.424, 655.545, 367.028, 424.114, 295.702, 558.76, 480.61, 459.416, 291.962, 391.503, 461.778, 644.39, 488.287, 425.033],
    'agw': [30.693, 49.118, 25.473, 29.237, 26.486, 28.547, 24.763, 23.882, 29.215, 28.785, 28.779, 26.71, 23.864, 25.387, 29.463, 28.609, 29.259, 20.889, 27.726, 27.658],
    'gn': [14488, 11556, 10389, 23492, 22889, 20322, 16820, 42889, 17564, 20569, 15636, 32545, 23712, 30183, 15980, 20807, 20719, 34173, 23543, 28911],
    'grain_yield': [2844.291, 2710.761, 2604.396, 3757.808, 2393.898, 3373.228, 4163.714, 6199.606, 2480.774, 3433.071, 2776.903, 5549.672, 5440.748, 4420.801, 2982.743, 3825.262, 4487.73, 6445.997, 4661.089, 4329.462]
}

# Splitting the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# RandomForest Model
rf = RandomForestRegressor()

# Training the model with cross-validation
scores = cross_val_score(rf, train_df.drop(columns='grain_yield'), train_df['grain_yield'], cv=4, n_jobs=-1, verbose=1)
print("Cross-validation scores:", scores)

# Fitting the model
rf.fit(train_df.drop(columns='grain_yield'), train_df['grain_yield'])

# Summarizing the model
importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': train_df.drop(columns='grain_yield').columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)
print(feature_importance_df)

# Predicting and calculating RMSE on training set
train_predictions = rf.predict(train_df.drop(columns='grain_yield'))
train_rmse = np.sqrt(mean_squared_error(train_df['grain_yield'], train_predictions))
print("Train RMSE:", train_rmse)

# Predicting and calculating RMSE on testing set
test_predictions = rf.predict(test_df.drop(columns='grain_yield'))
test_rmse = np.sqrt(mean_squared_error(test_df['grain_yield'], test_predictions))
print("Test RMSE:", test_rmse)

# Permutation importance
result = permutation_importance(rf, test_df.drop(columns='grain_yield'), test_df['grain_yield'], n_repeats=10, random_state=42, n_jobs=-1)
sorted_idx = result.importances_mean.argsort()
importance_df = pd.DataFrame({
    'feature': test_df.drop(columns='grain_yield').columns[sorted_idx],
    'importance': result.importances_mean[sorted_idx]
}).sort_values(by='importance', ascending=False)

print(importance_df)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.9s finished


Cross-validation scores: [-0.44272864  0.45719881  0.50338329  0.67537279]
        feature  importance
1  stem_biomass    0.659251
3            gn    0.175005
2           agw    0.108429
0        height    0.057315
Train RMSE: 253.68998788321832
Test RMSE: 429.47037330574216
        feature  importance
3  stem_biomass    0.924796
2            gn    0.100732
1           agw    0.068907
0        height    0.002780
